# Fine tuning LLM by FAQs on GST

In [1]:
!pip uninstall -y tensorflow --quiet
!pip install ludwig
!pip install ludwig[llm]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 16.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 682.2/682.2 kB 50.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.8/80.8 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 40.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.2/280.2 kB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.1/98.1 kB 14.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.0/232.0 kB 31.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 68.5 MB/s eta 0:

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 61.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.7/174.7 kB 23.1 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=6bc162815951c72c8879f6ff4e2f1d1798623be8a028c63e073409876aecbc49
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [2]:
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))

get_ipython().events.register('pre_run_cell', set_css)

def clear_cache():
  if torch.cuda.is_available():
    model = None
    torch.cuda.empty_cache()

In [3]:
import getpass
import locale; locale.getpreferredencoding = lambda: "UTF-8"
import logging
import os
import torch
import yaml

from ludwig.api import LudwigModel


os.environ["HUGGING_FACE_HUB_TOKEN"] = getpass.getpass("Token:")
assert os.environ["HUGGING_FACE_HUB_TOKEN"]

Token:··········


In [4]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir('/content/drive/MyDrive/ImpDocs/Work/AICoach/Notebooks/data')


Mounted at /content/drive


Do following only the first time.

In [5]:
# !pip install wget
# import wget

# # Replace the URL with the raw URL of the file on GitHub
# url = "https://raw.githubusercontent.com/yogeshhk/Sarvadnya/master/src/ludwig/data/cbic-gst_gov_in_fgaq.csv"

# # Download the file
# wget.download(url, 'cbic-gst_gov_in_fgaq.csv')

In [6]:
from google.colab import data_table; data_table.enable_dataframe_formatter()
import numpy as np; np.random.seed(123)
import pandas as pd

In [7]:
instruction_tuning_config_yaml = yaml.safe_load("""
model_type: llm
base_model: meta-llama/Llama-2-7b-hf

quantization:
 bits: 4

adapter:
 type: lora

prompt:
  template: |
    ### Instruction:
    You are a taxation expert on Goods and Services Tax used in India.
    Take the Input given below which is a Question. Give Answer for it as a Response.

    ### Input:
    {Question}

    ### Response:

input_features:
 - name: Question
   type: text

output_features:
 - name: Answer
   type: text

trainer:
 type: finetune
 learning_rate: 0.0003
 batch_size: 1
 gradient_accumulation_steps: 8
 epochs: 3

backend:
 type: local
""")

In [8]:
qna_tuning_config_dict = {
    "input_features": [
        {
            "name": "Question",
            "type": "text",
            "encoder": {
                "type": "auto_transformer",
                "pretrained_model_name_or_path": "meta-llama/Llama-2-7b-hf",
                "trainable": False
            },
            "preprocessing": {
                "cache_encoder_embeddings": True
            }
        }
    ],
    "output_features": [
        {
            "name": "Answer",
            "type": "text"
        }
    ]
}

In [9]:
df = pd.read_csv('/content/drive/MyDrive/ImpDocs/Work/AICoach/Notebooks/data/cbic-gst_gov_in_fgaq.csv', encoding='cp1252')
df.head()


,Question,Answer
0,Does aggregate turnover include value of inwar...,Refer Section 2(6) of CGST Act. Aggregate turn...
1,What if the dealer migrated with wrong PAN as ...,New registration would be required as partners...
2,A taxable person’s business is in many states....,He is liable to register if the aggregate turn...
3,Can we use provisional GSTIN or do we get new ...,Provisional GSTIN (PID) should be converted in...
4,Whether trader of country liquor is required t...,If the person is involved in 100% supply of go...


In [10]:
model = LudwigModel(config=instruction_tuning_config_yaml)
_ = model.train(dataset=df)

config.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

INFO:ludwig.utils.print_utils:
INFO:ludwig.utils.print_utils:╒════════════════════════╕
INFO:ludwig.utils.print_utils:│ EXPERIMENT DESCRIPTION │
INFO:ludwig.utils.print_utils:╘════════════════════════╛
INFO:ludwig.utils.print_utils:
INFO:ludwig.api:╒══════════════════╤═════════════════════════════════════════════════════════════════════════════════════════╕
│ Experiment name  │ api_experiment                                                                          │
├──────────────────┼─────────────────────────────────────────────────────────────────────────────────────────┤
│ Model name       │ run                                                                                     │
├──────────────────┼─────────────────────────────────────────────────────────────────────────────────────────┤
│ Output directory │ /content/drive/MyDrive/ImpDocs/Work/AICoach/Notebooks/data/results/api_experiment_run_0 │
├──────────────────┼─────────────────────────────────────────────────────────────────

tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

INFO:ludwig.trainers.trainer:Evaluation took 11.5299s

INFO:ludwig.utils.metrics_printed_table:╒═══════════════════════╤════════════╤══════════════╤════════════╕
│                       │      train │   validation │       test │
╞═══════════════════════╪════════════╪══════════════╪════════════╡
│ bleu                  │     0.0883 │       0.0608 │     0.0545 │
├───────────────────────┼────────────┼──────────────┼────────────┤
│ char_error_rate       │     2.0315 │       2.1007 │     1.9855 │
├───────────────────────┼────────────┼──────────────┼────────────┤
│ loss                  │     1.6914 │       1.9657 │     2.2095 │
├───────────────────────┼────────────┼──────────────┼────────────┤
│ next_token_perplexity │ 18506.5078 │   19569.5488 │ 19921.1934 │
├───────────────────────┼────────────┼──────────────┼────────────┤
│ perplexity            │ 31916.5234 │   31903.0742 │ 31941.6152 │
├───────────────────────┼────────────┼──────────────┼────────────┤
│ rouge1_fmeasure       │     0.32

DEBUG:ludwig.utils.checkpoint_utils:Saved checkpoint at /content/drive/MyDrive/ImpDocs/Work/AICoach/Notebooks/data/results/api_experiment_run_0/model/training_checkpoints/best.ckpt.
DEBUG:ludwig.utils.checkpoint_utils:Saved checkpoint at /content/drive/MyDrive/ImpDocs/Work/AICoach/Notebooks/data/results/api_experiment_run_0/model/training_checkpoints/latest.ckpt.
DEBUG:ludwig.trainers.trainer:Epoch 3 took: 1m 18.1810s.
DEBUG:ludwig.utils.checkpoint_utils:Saved checkpoint at /content/drive/MyDrive/ImpDocs/Work/AICoach/Notebooks/data/results/api_experiment_run_0/model/training_checkpoints/latest.ckpt.


Training: 100%|██████████| 324/324 [03:58<00:00,  1.36it/s, loss=0.206]

INFO:ludwig.utils.print_utils:
INFO:ludwig.utils.print_utils:╒═════════════════╕
INFO:ludwig.utils.print_utils:│ TRAINING REPORT │
INFO:ludwig.utils.print_utils:╘═════════════════╛
INFO:ludwig.utils.print_utils:
INFO:ludwig.api:╒══════════════════════════════╤════════════════════╕
│ Validation feature           │ Answer             │
├──────────────────────────────┼────────────────────┤
│ Validation metric            │ loss               │
├──────────────────────────────┼────────────────────┤
│ Best model step              │ 324                │
├──────────────────────────────┼────────────────────┤
│ Best model epoch             │ 3                  │
├──────────────────────────────┼────────────────────┤
│ Best model's validation loss │ 1.9656940698623657 │
├──────────────────────────────┼────────────────────┤
│ Best model's test loss       │ 2.2095017433166504 │
╘══════════════════════════════╧════════════════════╛
INFO:ludwig.api:
Finished: api_experiment_run
INFO:ludwig.api:Saved to

INFO:ludwig.utils.print_utils:
INFO:ludwig.utils.print_utils:╒══════════╕
INFO:ludwig.utils.print_utils:│ FINISHED │
INFO:ludwig.utils.print_utils:╘══════════╛
INFO:ludwig.utils.print_utils:


In [11]:
test_df = pd.DataFrame([
    {
        "Question": "What is GST?"
    },
    {
        "Question": "Does aggregate turnover include value of inward supplies received on which RCM is payable?"
    },
])

results = model.predict(dataset=test_df)
print(results)

DEBUG:ludwig.api:Preprocessing
DEBUG:ludwig.data.preprocessing:handle text features with prompt parameters
DEBUG:ludwig.data.preprocessing:build preprocessing parameters
DEBUG:ludwig.data.preprocessing:handle missing values
DEBUG:ludwig.data.preprocessing:cast columns
DEBUG:ludwig.data.preprocessing:build metadata
DEBUG:ludwig.data.preprocessing:build data
INFO:ludwig.utils.tokenizers:Loaded HuggingFace implementation of meta-llama/Llama-2-7b-hf tokenizer
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
DEBUG:ludwig.utils.strings_utils:max length of <built-in function format>: 73 < limit: 116
DEBUG:ludwig.api:Predicting


Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:ludwig.models.llm:Decoded text inputs for the first example in batch: ### instruction:
you are a taxation expert on goods and services tax used in india.
take the input given below which is a question. give answer for it as a response.

### input:
what is gst?

### response:

INFO:ludwig.models.llm:Decoded generated output for the first example in batch: ### instruction:
you are a taxation expert on goods and services tax used in india.
take the input given below which is a question. give answer for it as a response.

### input:
what is gst?

### response:
 nobody can give a better definition of gst than the constitution of india.
INFO:ludwig.models.llm:Decoded text inputs for the first example in batch: ### instruction:
you are a taxation expert on goods and services tax used in india.
take the input given below which is a question. give answer for it as a response.

### input:
does aggregate turnover include value of inward supplies received on which rcm is payable?

### respons

Prediction: 100%|██████████| 1/1 [00:06<00:00,  6.03s/it]


DEBUG:ludwig.api:Postprocessing
INFO:ludwig.utils.tokenizers:Loaded HuggingFace implementation of meta-llama/Llama-2-7b-hf tokenizer


(                                  Answer_predictions  \
0  [, nobody, can, give, a, better, definition, o...   
1  [, nobody, is, li, able, to, pay, r, cm, on, i...   

                                Answer_probabilities  \
0  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
1  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   

                                     Answer_response  Answer_probability  
0  [nobody can give a better definition of gst th...                -inf  
1  [nobody is liable to pay rcm on inward supplies.]                -inf  , 'results')


/usr/local/lib/python3.10/dist-packages/ludwig/features/feature_utils.py:102: RuntimeWarning: divide by zero encountered in log
  return np.sum(np.log(sequence_probabilities))
